In [12]:
'''Import packages for loading data, analysing, and plotting'''

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xesmf as xe
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy.ma as ma
import math
import os
import matplotlib.colors as colors
import scipy
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as FA
import mpl_toolkits.axisartist.grid_finder as GF

In [13]:
#Define PMIP generations and model names
pmip_v4='PMIP4'
pmip_v3='PMIP3'
pmip={}
pmip['PMIP4']=['ACCESS-ESM1-5',
               'AWI-ESM-1-1-LR',
               'CESM2',
               'CNRM-CM6-1',
               'EC-Earth3-LR',
               'FGOALS-f3-L',
               'FGOALS-g3',
               'GISS-E2-1-G',
               'HadGEM3-GC31-LL',
               'HadGEM3-GC31',
               'INM-CM4-8',
               'IPSL-CM6A-LR',
               'MIROC-ES2L',
               'MPI-ESM1-2-LR',
               'MRI-ESM2-0',
               'NESM3',
               'NorESM1-F',
               'NorESM2-LM',
               'UofT-CCSM-4']
pmip['PMIP3']=['BCC-CSM1-1',
               'CCSM4',
               'CNRM-CM5',
               'COSMOS-ASO',
               'CSIRO-Mk3L-1-2',
               'CSIRO-Mk3-6-0',
               'EC-EARTH-2-2',
               'FGOALS-g2',
               'FGOALS-s2',
               'GISS-E2-1-G',
               'GISS-E2-R',
               'HadCM3',
               'HadGEM2-CC',
               'HadGEM2-ES',
               'IPSL-CM5A-LR',
               'KCM1-2-2'
               'MIROC-ESM',
               'MPI-ESM-P',
               'MRI-CGCM3']

In [14]:
#No change needs here

'''Define calculating functions'''

#This function will get all available experiment names
def experimentlist():
    exps=[]
    file_path = "../data" 
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for d in dirnames:
            exps.append(d)
    return exps

#This function will get all available model names in the experiment 
def modellist(experiment_name):
    models=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for f in filenames:
            mname=f.split("_")[0]
            models.append(mname)
    return models

#This function will get all available filenames in the experiment 
def filenamelist(experiment_name):
    filenames=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, files in os.walk(file_path):
        for f in files:
            ff='../data/%s/%s'%(experiment_name,f)
            filenames.append(ff)
    return filenames

#This function will identify models in the ensemble
def identify_ensemble_members(variable_name,experiment_name):
    datadir="../data/%s" %(experiment_name)
    ensemble_members=!../scripts/find_experiment_ensemble_members.bash {experiment_name} {variable_name} {datadir}
    return ensemble_members

#This function will list excat model name
def extract_model_name(filename):
    file_no_path=filename.rpartition("/")
    file_strings=file_no_path[2].partition("_")
    model_name=file_strings[0]
    return model_name

def ensemble_members_dict(variable_name,experiment_name):
    ens_mems=identify_ensemble_members(variable_name,experiment_name)
    ens_mems_dict={extract_model_name(ens_mems[0]):ens_mems[0]}
    for mem in ens_mems[1:]:
        ens_mems_dict[extract_model_name(mem)]=mem
    return ens_mems_dict

experimentlist()

['midHolocene-cal-adj',
 'atl3',
 'lig127k',
 'midHolocene',
 'barchart',
 'lig127-cal-adj',
 'HadISST',
 'amm_atl3',
 'piControl',
 'GPCP',
 'C20-Reanalysis',
 'amm',
 'historical',
 '1pctCO2',
 'abrupt4xCO2',
 'midPliocene-eoi400',
 'observations',
 'lgm',
 'lgm-cal-adj',
 'PMIP3',
 'PMIP4',
 'PMIP3',
 'PMIP4']

In [4]:
modellist('reanalysis')

[]

In [5]:
modellist('piControl')

['GISS-E2-1-G',
 'FGOALS-g3',
 '.FGOALS-f3-L',
 '.CNRM-CM5',
 '.COSMOS-ASO',
 '.MIROC-ES2L',
 '.HadCM3',
 'HadGEM3-GC31-LL',
 '.CNRM-CM6-1',
 'INM-CM4-8',
 '.CSIRO-Mk3-6-0',
 'FGOALS-g2',
 '.EC-EARTH-2-2',
 'ACCESS-ESM1-5',
 'GISS-E2-R',
 'NorESM2-LM',
 '.EC-Earth3-LR',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 '.CESM2',
 '.CCSM4',
 '.BCC-CSM1-1',
 'MRI-ESM2-0',
 'UofT-CCSM-4',
 'IPSL-CM6A-LR',
 'AWI-ESM-1-1-LR',
 '.NESM3',
 'MPI-ESM1-2-LR',
 'KCM1-2-2',
 '.MRI-CGCM3',
 'HadGEM2-CC',
 'MPI-ESM-P',
 '.CSIRO-Mk3L-1-2',
 'NorESM1-F',
 'FGOALS-s2',
 'MIROC-ESM']

In [ ]:
modellist('historical')

In [ ]:
#*********CODE NOT RELEVANT FOR HADISST DUE TO DIFFERENT RESOLUTION TO CMIP MODELS --> REGRIDDING IS DIFFERENT*****************
#ensemble difference for HadISST
def ensemble_diffence(pmip_v,experiment_name,variable_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'HadISST')

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    #Set up the holding array
    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
 
    #loop over models
    n=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                modellist.append(gcm)
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                diff=expt_a-expt_b
                this_regridder=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                diff_1x1=this_regridder(diff) 
                expt_a_file.close()
                expt_b_file.close()
                data[n,:,:]=diff_1x1
                n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      

    return modellist

In [15]:
#ensemble difference for HadISST
def ensemble_diffence(pmip_v,experiment_name,variable_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'HadISST')

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    #Set up the holding array
    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
 
    #loop over models
    n=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                modellist.append(gcm)
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                this_regridder_a=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                expt_a_1x1=this_regridder_a(expt_a)
                this_regridder_b=xe.Regridder(expt_b_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                expt_b_1x1=this_regridder_b(expt_b)
                diff_1x1=expt_a_1x1-expt_b_1x1
                expt_a_file.close()
                expt_b_file.close()
                data[n,:,:]=diff_1x1
                n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      

    return modellist

In [16]:
#piControl - sst

#diff_sst_spatialmean_djf

filename='../outputs/netcdf/diff_sst/PMIP4_piControl_diff_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='sst_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_sst_spatialmean_jja
filename='../outputs/netcdf/diff_sst/PMIP4_piControl_diff_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='sst_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_sst_spatialmean_mam
filename='../outputs/netcdf/diff_sst/PMIP4_piControl_diff_sst_spatialmean_mam.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='sst_spatialmean_mam'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list


['ACCESS-ESM1-5',
 'CESM2',
 'CNRM-CM6-1',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31-LL',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']

In [ ]:
#piControl - AMM + AZM SST

#amm_pattern_mon

filename='../outputs/netcdf/diff_sst/PMIP4_piControl_diff_amm_pattern_mon.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='amm_pattern_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#atl3_pattern_mon

filename='../outputs/netcdf/diff_sst/PMIP4_piControl_diff_atl3_pattern_mon.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='atl3_pattern_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

In [17]:
#historical - sst

#diff_sst_spatialmean_djf

filename='../outputs/netcdf/diff_sst/PMIP4_historical_diff_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='sst_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_sst_spatialmean_jja

filename='../outputs/netcdf/diff_sst/PMIP4_historical_diff_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='sst_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_sst_spatialmean_jja

filename='../outputs/netcdf/diff_sst/PMIP4_historical_diff_sst_spatialmean_mam.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='sst_spatialmean_mam'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

['ACCESS-ESM1-5',
 'AWI-ESM-1-1-LR',
 'CESM2',
 'CNRM-CM6-1',
 'FGOALS-f3-L',
 'GISS-E2-1-G',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM2-LM']

In [ ]:
#hisotrical - AMM + AZM SST

#amm_pattern_mon

filename='../outputs/netcdf/diff_sst/PMIP4_historical_diff_amm_pattern_mon.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='amm_pattern_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#atl3_pattern_mon

filename='../outputs/netcdf/diff_sst/PMIP4_historical_diff_atl3_pattern_mon.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='atl3_pattern_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

In [18]:
#ensemble difference for GPCP spatialmean 
def ensemble_diffence(pmip_v,experiment_name,variable_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'GPCP')

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    #Set up the holding array
    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
 
    #loop over models
    n=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                modellist.append(gcm)
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                this_regridder_a=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                expt_a_1x1=this_regridder_a(expt_a)
                this_regridder_b=xe.Regridder(expt_b_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                expt_b_1x1=this_regridder_b(expt_b)
                diff_1x1=expt_a_1x1-expt_b_1x1
                expt_a_file.close()
                expt_b_file.close()
                data[n,:,:]=diff_1x1
                n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      

    return modellist

In [19]:
#piControl - pr

#diff_pr_spatialmean_djf

filename='../outputs/netcdf/diff_pr/PMIP4_piControl_diff_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_pr_spatialmean_jja
filename='../outputs/netcdf/diff_pr/PMIP4_piControl_diff_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_pr_spatialmean_mam
filename='../outputs/netcdf/diff_pr/PMIP4_piControl_diff_pr_spatialmean_mam.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_mam'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

['ACCESS-ESM1-5',
 'CESM2',
 'CNRM-CM6-1',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31-LL',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']

In [20]:
#historical - pr

#diff_pr_spatialmean_djf

filename='../outputs/netcdf/diff_pr/PMIP4_historical_diff_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_pr_spatialmean_jja
filename='../outputs/netcdf/diff_pr/PMIP4_historical_diff_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#diff_pr_spatialmean_mam
filename='../outputs/netcdf/diff_pr/PMIP4_historical_diff_pr_spatialmean_mam.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_mam'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

['ACCESS-ESM1-5',
 'AWI-ESM-1-1-LR',
 'CESM2',
 'CNRM-CM6-1',
 'FGOALS-f3-L',
 'GISS-E2-1-G',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM2-LM']

In [17]:
#ensemble difference for reanalysis amm+azm 
def ensemble_diffence(pmip_v,experiment_name,variable_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'C20-Reanalysis')

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    #Set up the holding array
    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
 
    #loop over models
    n=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                modellist.append(gcm)
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                this_regridder_a=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                expt_a_1x1=this_regridder_a(expt_a)
                this_regridder_b=xe.Regridder(expt_b_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
                expt_b_1x1=this_regridder_b(expt_b)
                diff_1x1=expt_a_1x1-expt_b_1x1
                expt_a_file.close()
                expt_b_file.close()
                data[n,:,:]=diff_1x1
                n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      

    return modellist

In [19]:
#piControl - AMM + AZM SST

#amm_pr_regression_mon

filename='../outputs/netcdf/diff_pr/PMIP4_piControl_diff_amm_pr_regression_mon.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='amm_pr_regression_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#atl3_pr_regression_mon

filename='../outputs/netcdf/diff_pr/PMIP4_piControl_diff_atl3_pr_regression_mon.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='atl3_pr_regression_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

['ACCESS-ESM1-5',
 'CESM2',
 'CNRM-CM6-1',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31-LL',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']

In [20]:
#hisotrical - AMM + AZM SST

#amm_pattern_mon

filename='../outputs/netcdf/diff_pr/PMIP4_historical_diff_amm_pr_regression_mon.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='amm_pr_regression_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

#atl3_pattern_mon

filename='../outputs/netcdf/diff_pr/PMIP4_historical_diff_atl3_pr_regression_mon.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='atl3_pr_regression_mon'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list

['ACCESS-ESM1-5',
 'AWI-ESM-1-1-LR',
 'CESM2',
 'CNRM-CM6-1',
 'FGOALS-f3-L',
 'GISS-E2-1-G',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM2-LM']